# Post-processing the simulation results of **M**

In this notebook, we process and visualise the outputted numerical results containning the total mass of solute  
**Structure dimensions:**  
- domain length: $L=11.4$ cm
- domain width : $W=7.3$ cm
- gain diameter: $d=0.92$ mm

**OpenFoam**  
The steady-state flow field is calculated using the modified _simpleHeleShaw_ solver in OpenFoam.  
**PTOF**  
The mean flow velocity is integrated while running the PTOF code: $\overline{u}=$  
The advective time unit: $\tau_a=\frac{\overline{u}}{d}=$

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

## 1. Define some _Functions_ for later reuse
- detect_file(caseDirectory, keyword): get file path that contains _keyword_ in the _caseDirectory_ folder
- breakthrough_time(caseDirectory): get the breakthrough time of the case in _caseDirectory_ folder
- read_file_full(file_path): read the .dat file 

In [3]:
#******** Automatically detect the file containing 'Keyword' and ending with '.dat'*************#
def detect_file(caseDirectory, keyword):
    """
    find and return the path to the first .dat file in `caseDirectory` containing `keyword` in the filename.
    """
    for filename in os.listdir(caseDirectory):
        if keyword in filename and filename.endswith('.dat'):
            file_path = os.path.join(caseDirectory, filename)
            print(f"Using file: {file_path}")
            return file_path
    print(f"No file found with '{keyword}' in the name.")
    exit() 

#******** Find the breakthrough time *************#
def breakthrough_time(caseDirectory):
    pd.set_option('display.unicode.east_asian_width', True) #solve the problem of out of order
    keyword="Data_absorption_time_M"
    file_path=detect_file(caseDirectory, keyword)
    #names=('Time','Tag','Mass','Patch')
    df = pd.read_table(file_path, sep=r'\s+')  # read data file
    print(df.head())       #output the first 5 line
    Time=np.asarray(df['Time'])
    Mass=np.asarray(df['Mass'])
    # Sort outlet data by Time
    sorted_indices = np.argsort(Time)    #returns the indices of the sorted array
    Time_sorted = Time[sorted_indices]   #new time array sorted
    Mass_sorted = Mass[sorted_indices]   #new time array sorted
    breakthrough_time = Time_sorted[0]
    print(f"Breakthrough time: t= {breakthrough_time}")
    return breakthrough_time

#******** Read the detected full file, convert column data into arrarys*************#
def read_file(file_path, BTtime):
    pd.set_option('display.unicode.east_asian_width', True) #solve the problem of out of order
    #names=('Time','Mass') 
    df = pd.read_table(file_path, sep=r'\s+')  # read data file
    print(df.head())       #output the first 5 line
    time_full=np.asarray(df['Time'])
    mass_full=np.asarray(df['Mass'])

    df_filtered=df[df['Time'] < BTtime]   #filter the data before breakthrough 
    time_BT=np.asarray(df_filtered['Time'])
    mass_BT=np.asarray(df_filtered['Mass'])
    return time_full, mass_full, time_BT, mass_BT

## 2. Solid surface decay

We apply first-order surface decay for the grain surface, then solve the advection_diffusion_reaction equations for particles

## 3. Gas surface decay

We simply apply the **reacting** boundary condition to the bubble surface.

In [4]:
keyword="Data_mass_M"

#----------------Solid surface Pe1e2 Da1e0-----------------------------------# 
So_Pe1e2_Da1e0="/mnt/disk2/OPTOD_ARC_NEW/n13/1_Surface_decay_2d/Particle1e5/Pe1e2/n13_Da_1e0/output"     #case folder
So_Pe1e2_Da1e0_BTtime=breakthrough_time(So_Pe1e2_Da1e0)                                                  #get breakthrough time
So_Pe1e2_Da1e0_Filepath=detect_file(So_Pe1e2_Da1e0, keyword)                                             #get 
So_Pe1e2_Da1e0_Time, Soli_Pe1e2_Da1e0_Mass, So_Pe1e2_Da1e0_time_BT, Soli_Pe1e2_Da1e0_mass_BT=read_file(Soli_Pe1e2_Da1e0_Filepath, So_Pe1e2_Da1e0_BTtime)


Using file: /mnt/disk2/OPTOD_ARC_NEW/n13/1_Surface_decay_2d/Particle1e5/Pe1e2/n13_Da_1e0/output/Data_absorption_time_M_advection_diffusion_surface_decay_2d_C_n13_Da_1e0_OF_flow_b_T_Pe_1e2_R_Da_1e0_S_1e-1_1e-3_1e5_I_fluxweighted_O_moments_breakthrough_mass_1e-3_RUN_0.dat
          Time  Tag          Mass
0   8899.44112    0  7.582867e-08
1  10680.34040    1  1.421157e-08
2   8862.50099    2  1.868734e-08
3  12797.05840    3  4.072471e-10
4  19984.36740    4  1.585163e-11
Breakthrough time: t= 4613.14572
Using file: /mnt/disk2/OPTOD_ARC_NEW/n13/1_Surface_decay_2d/Particle1e5/Pe1e2/n13_Da_1e0/output/Data_mass_M_advection_diffusion_surface_decay_2d_C_n13_Da_1e0_OF_flow_b_T_Pe_1e2_R_Da_1e0_S_1e-1_1e-3_1e5_I_fluxweighted_O_moments_breakthrough_mass_1e-3_RUN_0.dat


NameError: name 'Soli_Pe1e2_Da1e0_Filepath' is not defined

In [5]:
#******** Plot data*************#
plt.plot(Soli_Pe1e2_Da1e0_Time, Soli_Pe1e2_Da1e0_Mass, color='red', linestyle='-')
plt.plot(Soli_Pe1e2_Da1e0_Time, Soli_Pe1e2_Da1e0_Mass, color='red', linestyle='-')

#******************* Set font and legend ***********************************************#
# Set global font to 'Liberation Serif'
rc = {"font.family": "serif", "mathtext.fontset": "cm"}
plt.rcParams.update(rc)
plt.rcParams['font.family'] = ['serif']
plt.rcParams["font.serif"] = ["Liberation Serif"]
# Set specific font properties for labels and legends
font_label = {'family': 'Liberation Serif', 'size': 15, 'style': 'italic'}
font_legend = {'family': 'Liberation Serif', 'size': 10, 'style': 'italic'}

plt.xlabel(r'$t$', font_label)
plt.ylabel(r'$M$', font_label)
plt.xlim(0,5000)
#plt.ylim(0,1.01)
#plt.xscale('log')
#plt.yscale('log')
#plt.grid(axis='y',linestyle=':')
#plt.gca().set_aspect(1)
#plt.legend(['old_code','new_code'],loc='lower left',markerscale=1,prop=font_legend,frameon=True)
plt.savefig('Mass.png', dpi=500, bbox_inches='tight', format='png')
#plt.savefig('Draw_caviy_depth.pdf', dpi=500, bbox_inches='tight', format='pdf')
#plt.show()

NameError: name 'Soli_Pe1e2_Da1e0_Time' is not defined

## 4. Reverssible gas surface reaction